In [2]:
import requests
import json
import pandas as pd

def fetch(ticker,bc_api_endpoint,apikey,FROM,TO):
    #httpのGETメソッド
    response = requests.get(
                            #アクセスしたurlを取得
                            url=bc_api_endpoint,
                            #urlパラメータを指定
                            params={
                                    #銘柄コードを指定
                                    "tickers":ticker,
                                    #期間を指定
                                    "from":FROM,
                                    "to":TO,
                                    },
                                    #レスポンスヘッダーを指定
                                    headers={
                                           #apikeyを指定
                                            "x-api-key":apikey,
                                            },
                            )
    return response

In [ ]:
#<<四半期の財務数値>>

#urlの最後の" / "の後ろをquarterと指定
bc_api_endpoint = "https://api.buffett-code.com/api/v2/quarter"

#自分のAPIキーの文字列を挿入
apikey =  "YourApiKey" 

#期間の指定
START_q ="2019Q1" #2019年度の第1四半期
END_q ="2019Q4"   #2019年度の第4四半期

#最大3つまで銘柄を一気に指定できる (例 1301,1332,1333 )
ticker_code = input()
#" , "を頼りに銘柄を１個ずつに分けリスト化
ticker = ticker_code.split(",")

res_q = fetch(ticker_code,bc_api_endpoint,apikey,START_q,END_q)
json_data_q = json.loads(res_q.text)
#バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
df_q_0 = pd.DataFrame.from_dict(json_data_q[ticker[0]]) #リストの0行目の銘柄
df_q_1 = pd.DataFrame.from_dict(json_data_q[ticker[1]]) #リストの1行目の銘柄
df_q_2 = pd.DataFrame.from_dict(json_data_q[ticker[2]]) #リストの2行目の銘柄
#リスト化をする
df_q_list = [df_q_0,df_q_1,df_q_2]
#確認用
df_q_list[2]

In [ ]:
#<<日別の株価指標>>
#urlの最後の" / "の後ろをdailyと指定
bc_api_endpoint = "https://api.buffett-code.com/api/v2/daily"
apikey = "YourApiKey" 

START_day = "2019-04-01"#2019年度の第1四半期の時系列と合わせる
END_day = "2020-03-31"  #2019年度の第4四半期の時系列と合わせる

ticker_code = input()
ticker = ticker_code.split(",")

res_day = fetch(ticker_code, bc_api_endpoint, apikey, START_day, END_day) 
json_data_day = json.loads(res_day.text)    

#バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
df_d_0 = pd.DataFrame.from_dict(json_data_day[ticker[0]])
df_d_1 = pd.DataFrame.from_dict(json_data_day[ticker[1]])
df_d_2 = pd.DataFrame.from_dict(json_data_day[ticker[2]])
df_d_list = [df_d_0,df_d_1,df_d_2]
df_d_list[2]

In [ ]:
from datetime import *
#urlの最後の" / "の後ろをdailyと指定
bc_api_endpoint = "https://api.buffett-code.com/api/v2/daily"
apikey = "YourApiKey" 
year = 2019 #START_dayの一部をyearに代入
month = 4  #START_dayの一部をmonthに代入
START_day = f"{year}-0{month}-01" #2019年度の第1四半期の時系列と合わせる
END_day =  "2020-03-31" #2019年度の第4四半期の時系列と合わせる

ticker_code = input()
ticker = ticker_code.split(",")
res_day = fetch(ticker_code, bc_api_endpoint, apikey, START_day, END_day) 
json_data_day = json.loads(res_day.text)    
df_d_0 = pd.DataFrame.from_dict(json_data_day[ticker[0]])
df_d_1 = pd.DataFrame.from_dict(json_data_day[ticker[1]])
df_d_2 = pd.DataFrame.from_dict(json_data_day[ticker[2]])
df_d_list = [df_d_0,df_d_1,df_d_2]
df_d_list[2]
#4分割し平均化したデータをリスト化する
df_day_list = []
for i in range(len(df_d_list)):
    #datetimeに変換させ時間の比較を行うようにする
    df_d_list[i]['to_datetime'] = pd.to_datetime(df_d_list[i]['day'])

    #mergeのための、空フレームを作成
    df_merge = pd.DataFrame()
    #四半期ごとに分割するために、最初の検索月を指定する際などに利用
    searchConditionMonth = month - 1 
    #四半期ごとに分割するために、+3か月して値を更新する際などに利用
    searchConditionMonthIncrement = 3 
    #yearを代入
    searchConditionYear = year 

    count = 0
    while True:
        count += 1
        #4回分割したら終了(1年分）
        if 4 + 1 <= count:
            break
            
        # 検索条件（From）～年～月１日より期間スタート
        searchConditionDatetimeFrom = datetime(searchConditionYear, searchConditionMonth + 1, 1, tzinfo=timezone.utc)

        # 検索期間の月と年を更新する
        searchConditionMonth = (searchConditionMonth + searchConditionMonthIncrement) % 12
        # 12で割ったあまりが1(13月 = 10月 + 3月)になった際、年を更新させる
        if searchConditionMonth < searchConditionMonthIncrement:
            searchConditionYear += 1

        # 検索条件（To）～年～(+3)月 で期間をストップ
        searchConditionDatetimeTo = datetime(searchConditionYear, searchConditionMonth + 1, 1, tzinfo=timezone.utc)   
        
        #検索期間内にあるデータを抽出
        df_dq = df_d_list[i][(searchConditionDatetimeFrom <= df_d_list[i]['to_datetime']) &
                                                            (df_d_list[i]['to_datetime'] < searchConditionDatetimeTo)
                            ]
        #四半期ごとのデータを平均化しmergeさせる
        df_dq_mean = df_dq.mean()
        df_DQ = pd.DataFrame(df_dq_mean)
        df_merge = pd.concat([df_merge,df_DQ],axis = 1)

    #4分割し平均化したデータをリスト化する
    df_day_list.append(df_merge.T) #day
#確認用
df_day_list[2] 

In [ ]:
#<<直近営業日の株価指標>>

#urlの最後の" / "の後ろをindicatorと指定
bc_api_endpoint = "https://api.buffett-code.com/api/v2/indicator"
apikey = "YourApiKey"

ticker_code = input()
ticker = ticker_code.split(",")
#期間を指定せず、時点(直近営業日)を指定しいているため"None"を記入
res_c = fetch(ticker_code,bc_api_endpoint,apikey,None,None)
json_data_c = json.loads(res_c.text)
#バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
df_c_0 = pd.DataFrame.from_dict(json_data_c[ticker[0]]) 
df_c_1 = pd.DataFrame.from_dict(json_data_c[ticker[1]]) 
df_c_2 = pd.DataFrame.from_dict(json_data_c[ticker[2]]) 
df_c_list = [df_c_0,df_c_1,df_c_2]
#確認用
df_c_list[1]

In [3]:
import requests
import json
import pandas as pd
#fetch関数を確認のためコピペ(コピペしなくてもよい)
def fetch(ticker,bc_api_endpoint,apikey,FROM,TO):

    response = requests.get(
                            url=bc_api_endpoint,
                            params={
                                    "tickers":ticker,
                                    "from":FROM,
                                    "to":TO,
                                    },
                                    headers={
                                            "x-api-key":apikey,
                                            },
                            )
    return response

In [4]:
from datetime import *
def make_df(ticker_code:str):
    ticker = ticker_code.split(",")
    ####  四半期の財務数値をコピペ #######################################
    bc_api_endpoint = "https://api.buffett-code.com/api/v2/quarter"
    #自分のAPIキーの文字列を挿入
    apikey = "YourApiKey" 

    #期間の指定
    START_q ="2019Q1" #2019年度の第1四半期
    END_q ="2019Q4"   #2019年度の第4四半期

    #最大3つまで銘柄を一気に指定できる (例 1301,1332,1333 )
#     ticker_code = input()
    #" , "を頼りに銘柄を１個ずつに分けリスト化
#     ticker = ticker_code.split(",")

    res_q = fetch(ticker_code,bc_api_endpoint,apikey,START_q,END_q)
    json_data_q = json.loads(res_q.text)
    #バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
    df_q_0 = pd.DataFrame.from_dict(json_data_q[ticker[0]]) #リストの0行目の銘柄
    df_q_1 = pd.DataFrame.from_dict(json_data_q[ticker[1]]) #リストの1行目の銘柄
    df_q_2 = pd.DataFrame.from_dict(json_data_q[ticker[2]]) #リストの2行目の銘柄
    #リスト化をする
    df_q_list = [df_q_0,df_q_1,df_q_2]
    #確認用
#     df_q_list[2]

    ##### 日別の株価指標をコピペ #########################################
    #datetimeをインポート
#     from datetime import *
    #urlの最後の" / "の後ろをdailyと指定
    bc_api_endpoint = "https://api.buffett-code.com/api/v2/daily"
#     apikey = "YourApiKey"  APIキーを上で定義している

    year = 2019 #START_dayの一部をyearに代入
    month = 4  #START_dayの一部をmonthに代入
    START_day = f"{year}-0{month}-01" #2019年度の第1四半期の時系列と合わせる
    END_day =  "2020-03-31" #2019年度の第4四半期の時系列と合わせる

#     ticker_code = input()
#    ticker = ticker_code.split(",") 

    res_day = fetch(ticker_code, bc_api_endpoint, apikey, START_day, END_day) 
    json_data_day = json.loads(res_day.text)    

    #バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
    df_d_0 = pd.DataFrame.from_dict(json_data_day[ticker[0]])
    df_d_1 = pd.DataFrame.from_dict(json_data_day[ticker[1]])
    df_d_2 = pd.DataFrame.from_dict(json_data_day[ticker[2]])
    df_d_list = [df_d_0,df_d_1,df_d_2]
    df_d_list[2]


    #4分割し平均化したデータをリスト化する
    df_day_list = []
    for i in range(len(df_d_list)):
        #datetimeに変換させ時間の比較を行うようにする
        df_d_list[i]['to_datetime'] = pd.to_datetime(df_d_list[i]['day'])

        #mergeのための、空フレームを作成
        df_merge = pd.DataFrame()
        #四半期ごとに分割するために、最初の検索月を指定する際などに利用
        searchConditionMonth = month - 1 
        #四半期ごとに分割するために、+3か月して値を更新する際などに利用
        searchConditionMonthIncrement = 3 
        #yearを代入
        searchConditionYear = year 

        count = 0
        while True:
            count += 1
            #4回分割したら終了(1年分）
            if 4 + 1 <= count:
                break

            # 検索条件（From）～年～月１日より期間スタート
            searchConditionDatetimeFrom = datetime(searchConditionYear, searchConditionMonth + 1, 1, tzinfo=timezone.utc)

            # 検索期間の月と年を更新する
            searchConditionMonth = (searchConditionMonth + searchConditionMonthIncrement) % 12
            # 12で割ったあまりが1(13月 = 10月 + 3月)になった際、年を更新させる
            if searchConditionMonth < searchConditionMonthIncrement:
                searchConditionYear += 1

            # 検索条件（To）～年～(+3)月 で期間をストップ
            searchConditionDatetimeTo = datetime(searchConditionYear, searchConditionMonth + 1, 1, tzinfo=timezone.utc)   

            #検索期間内にあるデータを抽出
            df_dq = df_d_list[i][(searchConditionDatetimeFrom <= df_d_list[i]['to_datetime']) &
                                                                (df_d_list[i]['to_datetime'] < searchConditionDatetimeTo)
                                ]
            #四半期ごとのデータを平均化しmergeさせる
            df_dq_mean = df_dq.mean()
            df_DQ = pd.DataFrame(df_dq_mean)
            df_merge = pd.concat([df_merge,df_DQ],axis = 1)

        #4分割し平均化したデータをリスト化する
        df_day_list.append(df_merge.T) #day
    #確認用
#     df_day_list[2]   
    ######  直近営業日の株価指標をコピペ  ################################################
    bc_api_endpoint = "https://api.buffett-code.com/api/v2/indicator"
#     apikey = "YourApiKey"

#     ticker_code = input()
#     ticker = ticker_code.split(",") 
    #期間を指定せず、時点(直近営業日)を指定しいているため"None"を記入
    res_c = fetch(ticker_code,bc_api_endpoint,apikey,None,None)
    json_data_c = json.loads(res_c.text)
    #バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
    df_c_0 = pd.DataFrame.from_dict(json_data_c[ticker[0]]) 
    df_c_1 = pd.DataFrame.from_dict(json_data_c[ticker[1]]) 
    df_c_2 = pd.DataFrame.from_dict(json_data_c[ticker[2]]) 
    df_c_list = [df_c_0,df_c_1,df_c_2]
    #確認用
#     df_c_list[1]
    return df_q_list,df_day_list,df_c_list

In [ ]:
ticker_code = input()
df_q_list,df_day_list,df_c_list = make_df(ticker_code)
df_q_list[2]

In [ ]:
df_day_list[2]

In [ ]:
df_c_list[2]